In [1]:
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from datetime import datetime
from time import sleep, strftime, localtime  
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import pandas as pd
import backtrader as bt
import pytz, tzlocal
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
handler = open('fd.txt').read()
parser = BeautifulSoup(handler, 'lxml')

In [3]:
parser

<html><body><p>'<financialsummary>\n   <dividendpershares currency="USD">\n      <dividendpershare asofdate="2019-06-30" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2019-03-31" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2018-12-31" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2018-09-30" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2018-06-30" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2018-03-31" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2017-12-31" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2017-09-30" period="12M" reporttype="TTM">0.000000</dividendpershare>\n      <dividendpershare asofdate="2017-06-30" period="12M" reporttype="TTM">0.000000<

In [24]:
date_div = []
data_div = []
for i in parser.findAll('dividendpershare'):
    if i['period'] == '12M' and i['reporttype'] == 'TTM':
        date_div.append(i['asofdate'])
        data_div.append(float(i.text))

data_rev = []
for i in parser.findAll('totalrevenue'):
    if i['period'] == '12M' and i['reporttype'] == 'TTM':
        data_rev.append(float(i.text))

data_eps = []
for i in parser.findAll('eps'):
    if i['period'] == '12M' and i['reporttype'] == 'TTM':
        data_eps.append(float(i.text))


fin = pd.DataFrame(index = date_div,
                       data = {'Dividend Per Share TTM':data_div,'Total Revenue TTM':data_rev, 'EPS TTM':data_eps}).sort_index(axis = 0,ascending = False)
 
fin.head()    


,Dividend Per Share TTM,EPS TTM,Total Revenue TTM
2019-06-30,0.0,0.09,271041000.0
2019-03-31,0.0,0.02,268398000.0
2018-12-31,0.0,-0.08,262841000.0
2018-09-30,0.0,-0.15,247868000.0
2018-06-30,0.0,-0.15,227269000.0


In [15]:
parser.findAll('sector')[0]

<sector code="56" set="RBSS2004">Healthcare</sector>

In [188]:
annual = []
columns = []
quarter = []


In [189]:
for i,item in enumerate(parser.findAll('fyactual')):
    columns.append(item['type'])
    for per in item.findAll('fyperiod'):
        if per['periodtype'] == 'A':
            annual.append(per['fyear'])
        if per['periodtype'] == 'Q':
            quarter.append('{}-{}'.format(per['endcalyear'],per['endmonth']))


In [190]:
index = list(set(annual))
index_q = list(set(quarter))

In [191]:
resc_ann = pd.DataFrame(index = index, columns = columns).sort_index(axis = 0,ascending = False)
resc_q = pd.DataFrame(index = index_q, columns = columns).sort_index(axis = 0,ascending = False)

In [192]:
resc_ann

,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019
2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
resc_q.tail()

,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019
2014-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
#datetime.strptime('2018-09', '%Y-%m').strftime('%Y-%m')

In [181]:
resc_q

TypeError: sort_index() got an unexpected keyword argument 'reverse'